In [21]:
import aif360
print("AIF360 is working!")


AIF360 is working!


In [23]:
import pandas as pd
import numpy as np

# AIF360 core components
from aif360.datasets import BinaryLabelDataset
from aif360.metrics import ClassificationMetric
from aif360.algorithms.postprocessing import RejectOptionClassification


In [28]:
import os
print(os.getcwd())




C:\Users\Gille\Downloads\PostProcess


In [31]:
df_1 = pd.read_csv("predictions_binary.csv")
df_1.head()


,row_index,true_value,regression_beat,regression_grf_noZ,regression_grf_all,regression_xgboost,regression_beat_log,regression_logistic,sex_Male,age_cat_25 - 45,age_cat_Greater than 45,age_cat_Less than 25,race_African-American,race_Asian,race_Caucasian,race_Hispanic,race_Native American,race_Other
0,13117,0,0,0,0,0,0,0,1,1,0,0,0,0,0,1,0,0
1,13118,0,0,0,0,0,0,0,1,0,1,0,0,0,1,0,0,0
2,13119,1,1,1,1,1,0,1,1,1,0,0,0,0,1,0,0,0
3,13120,0,1,1,1,1,1,1,1,1,0,0,1,0,0,0,0,0
4,13121,0,1,1,1,1,1,1,1,1,0,0,1,0,0,0,0,0


In [32]:
df_2 = pd.read_csv("predictions_scores.csv")
df_2.head()

,row_index,true_value,regression_beat,regression_grf_noZ,regression_grf_all,regression_xgboost,regression_beat_log,regression_logistic,sex_Male,age_cat_25 - 45,age_cat_Greater than 45,age_cat_Less than 25,race_African-American,race_Asian,race_Caucasian,race_Hispanic,race_Native American,race_Other
0,13117,0,0.292985,0.148446,0.178030,0.099230,0.142576,0.268147,1,1,0,0,0,0,0,1,0,0
1,13118,0,0.229411,0.128625,0.103476,0.121780,0.226369,0.257027,1,0,1,0,0,0,1,0,0,0
2,13119,1,0.504424,0.797536,0.663396,0.574116,0.491491,0.535090,1,1,0,0,0,0,1,0,0,0
3,13120,0,0.641172,0.868008,0.857357,0.924456,0.667642,0.848925,1,1,0,0,1,0,0,0,0,0
4,13121,0,0.641172,0.868008,0.857357,0.924456,0.667642,0.848925,1,1,0,0,1,0,0,0,0,0


In [33]:
score_columns = ["regression_beat","regression_grf_noZ","regression_grf_all",
     "regression_xgboost","regression_beat_log","regression_logistic"] 

#Making sure the probabilities reflect chance of the favourable outcome which is in our case non-recidivism
df_2[score_columns] = 1 - df_2[score_columns]
df_2.head()

,row_index,true_value,regression_beat,regression_grf_noZ,regression_grf_all,regression_xgboost,regression_beat_log,regression_logistic,sex_Male,age_cat_25 - 45,age_cat_Greater than 45,age_cat_Less than 25,race_African-American,race_Asian,race_Caucasian,race_Hispanic,race_Native American,race_Other
0,13117,0,0.707015,0.851554,0.821970,0.900770,0.857424,0.731853,1,1,0,0,0,0,0,1,0,0
1,13118,0,0.770589,0.871375,0.896524,0.878220,0.773631,0.742973,1,0,1,0,0,0,1,0,0,0
2,13119,1,0.495576,0.202464,0.336604,0.425884,0.508509,0.464910,1,1,0,0,0,0,1,0,0,0
3,13120,0,0.358828,0.131992,0.142643,0.075544,0.332358,0.151075,1,1,0,0,1,0,0,0,0,0
4,13121,0,0.358828,0.131992,0.142643,0.075544,0.332358,0.151075,1,1,0,0,1,0,0,0,0,0


In [34]:
df_1[df_1['age_cat_Greater than 45'] == 1][['true_value', 'regression_logistic']].mean()

true_value             0.344907
regression_logistic    0.239583
dtype: float64

In [35]:
df_1[df_1['sex_Male'] == 1][['true_value', 'regression_logistic']].mean()


true_value             0.498016
regression_logistic    0.523526
dtype: float64

In [36]:
df_1[df_1['race_African-American'] == 1][['true_value', 'regression_logistic']].mean()


true_value             0.536337
regression_logistic    0.611560
dtype: float64

In [37]:
prediction_cols = ["regression_beat", "regression_grf_all","regression_beat_log",
                   "regression_xgboost","regression_logistic","regression_grf_noZ"]

protected_info = {
    "race_African-American": {"unpriv": 1, "priv": 0},
    "sex_Male": {"unpriv": 1, "priv": 0 },
    "age_cat_Greater than 45" : {"unpriv": 0, "priv": 1}
    }



In [51]:
def get_fairness_metric_table(df, prediction_cols, protected_info, true_label_col="true_value"):
    results = []

    for attr, group_vals in protected_info.items():
        unpriv_val = group_vals["unpriv"]
        priv_val = group_vals["priv"]

        for pred_col in prediction_cols:
            df_base = df.copy()

            # Create ground truth dataset
            gt_dataset = BinaryLabelDataset(
                favorable_label= 0,
                unfavorable_label= 1,
                df=df_base,
                label_names=[true_label_col],
                protected_attribute_names=[attr]
            )

            # Replace label column with prediction
            df_base[true_label_col] = df_base[pred_col]

            # Create prediction dataset
            pred_dataset = BinaryLabelDataset(
                favorable_label= 0,
                unfavorable_label= 1,
                df=df_base,
                label_names=[true_label_col],
                protected_attribute_names=[attr]
            )

            # Group mapping
            unprivileged_groups = [{attr: unpriv_val}]
            privileged_groups = [{attr: priv_val}]

            # Metrics
            metric = ClassificationMetric(
                gt_dataset,
                pred_dataset,
                unprivileged_groups=unprivileged_groups,
                privileged_groups=privileged_groups
            )

            # Store results
            results.append({
                "protected_attribute": attr,
                "prediction_column": pred_col,
                 "version": "not-postprocessed",
                "accuracy": metric.accuracy(),
                "disparate_impact": metric.disparate_impact(),
                "equal_opportunity_diff": metric.equal_opportunity_difference(),
                "average_odds_diff": metric.average_odds_difference()
            })

    # Convert to DataFrame
    return pd.DataFrame(results)



In [52]:
df_not = get_fairness_metric_table(df_1, prediction_cols, protected_info)
df_not.head(20)

,protected_attribute,prediction_column,version,accuracy,disparate_impact,equal_opportunity_diff,average_odds_diff
0,race_African-American,regression_beat,not-postprocessed,0.635042,0.726800,-0.119929,-0.127492
1,race_African-American,regression_grf_all,not-postprocessed,0.666436,0.659700,-0.195472,-0.192152
2,race_African-American,regression_beat_log,not-postprocessed,0.632964,0.736708,-0.126916,-0.150479
3,race_African-American,regression_xgboost,not-postprocessed,0.614958,0.748884,-0.178313,-0.160276
4,race_African-American,regression_logistic,not-postprocessed,0.652355,0.540593,-0.291251,-0.298779
5,race_African-American,regression_grf_noZ,not-postprocessed,0.649123,0.686448,-0.218416,-0.180964
6,sex_Male,regression_beat,not-postprocessed,0.635042,0.863831,-0.088283,-0.045864
7,sex_Male,regression_grf_all,not-postprocessed,0.666436,0.691025,-0.188817,-0.200470
8,sex_Male,regression_beat_log,not-postprocessed,0.632964,0.872122,-0.081809,-0.056861
9,sex_Male,regression_xgboost,not-postprocessed,0.614958,0.858429,-0.086825,-0.082710


In [40]:
def get_roc_fairness_metrics_only(
    df,
    prediction_cols,
    protected_info,
    true_label_col="true_value",
    metric_name="Average odds difference",
    metric_lb=-0.05,
    metric_ub=0.05
):
    results = []

    for attr, group_vals in protected_info.items():
        unpriv_val = group_vals["unpriv"]
        priv_val = group_vals["priv"]

        for pred_col in prediction_cols:
            # Ground truth dataset
            gt_dataset = BinaryLabelDataset(
                favorable_label= 0,
                unfavorable_label=1,
                df=df,
                label_names=[true_label_col],
                protected_attribute_names=[attr]
            )

            # Create dataset with raw scores (dummy label used)
            df_prob = df.copy()
            df_prob["true_value"] = 0

            pred_ds_probs = BinaryLabelDataset(
                favorable_label= 0,
                unfavorable_label= 1,
                df=df_prob,
                label_names=["true_value"],
                protected_attribute_names=[attr]
            )

            # Add scores
            pred_ds_probs.scores = df[pred_col].values.reshape(-1, 1)

            # Define group mapping
            unprivileged_groups = [{attr: unpriv_val}]
            privileged_groups = [{attr: priv_val}]

            # Apply ROC
            roc = RejectOptionClassification(
                unprivileged_groups=unprivileged_groups,
                privileged_groups=privileged_groups,
                low_class_thresh=0.01,
                high_class_thresh=0.99,
                num_class_thresh=100,
                num_ROC_margin=50,
                metric_name=metric_name,
                metric_ub=metric_ub,
                metric_lb=metric_lb
            )

            roc.fit(gt_dataset, pred_ds_probs)
            pred_ds_adj = roc.predict(pred_ds_probs)

            # Evaluate post-ROC fairness
            metric_adj = ClassificationMetric(
                gt_dataset,
                pred_ds_adj,
                unprivileged_groups=unprivileged_groups,
                privileged_groups=privileged_groups
            )

            results.append({
                "protected_attribute": attr,
                "prediction_column": pred_col,
                "version": "postprocessed",
                "accuracy": metric_adj.accuracy(),
                "disparate_impact": metric_adj.disparate_impact(),
                "equal_opportunity_diff": metric_adj.equal_opportunity_difference(),
                "average_odds_diff": metric_adj.average_odds_difference()
            })

    return pd.DataFrame(results)



In [41]:
df_post = get_roc_fairness_metrics_only(df_2, prediction_cols, protected_info)
df_post.head(20)

,protected_attribute,prediction_column,version,accuracy,disparate_impact,equal_opportunity_diff,average_odds_diff
0,race_African-American,regression_beat,postprocessed,0.634811,0.839206,-0.046149,-0.041449
1,race_African-American,regression_grf_all,postprocessed,0.656971,0.850834,-0.048031,-0.040344
2,race_African-American,regression_beat_log,postprocessed,0.641043,0.864365,-0.039437,-0.027242
3,race_African-American,regression_xgboost,postprocessed,0.619344,0.956032,-0.020246,-0.001231
4,race_African-American,regression_logistic,postprocessed,0.660203,0.905257,-0.042383,-0.017678
5,race_African-American,regression_grf_noZ,postprocessed,0.645429,0.891209,-0.069574,-0.036966
6,sex_Male,regression_beat,postprocessed,0.638735,0.844233,-0.099366,-0.044573
7,sex_Male,regression_grf_all,postprocessed,0.669437,0.922934,-0.037687,-0.011372
8,sex_Male,regression_beat_log,postprocessed,0.640120,0.837749,-0.099379,-0.046308
9,sex_Male,regression_xgboost,postprocessed,0.623730,0.928522,-0.033787,-0.027753


In [16]:
df_post.head(20)

,protected_attribute,prediction_column,version,accuracy,disparate_impact,equal_opportunity_diff,average_odds_diff
0,race_African-American,regression_beat,postprocessed,0.632271,0.929175,-0.001858,0.006220
1,race_African-American,regression_grf_all,postprocessed,0.654201,0.830584,-0.059070,-0.045136
2,race_African-American,regression_beat_log,postprocessed,0.645199,0.843482,-0.038786,-0.048665
3,race_African-American,regression_xgboost,postprocessed,0.625808,0.967144,-0.032773,0.011183
4,race_African-American,regression_logistic,postprocessed,0.658587,0.866217,-0.064667,-0.035786
5,race_African-American,regression_grf_noZ,postprocessed,0.644044,0.867754,-0.085364,-0.044496
6,sex_Male,regression_beat,postprocessed,0.638735,0.878753,-0.077515,-0.025810
7,sex_Male,regression_grf_all,postprocessed,0.664589,0.895601,-0.038922,-0.022911
8,sex_Male,regression_beat_log,postprocessed,0.651200,0.887269,-0.069308,-0.034999
9,sex_Male,regression_xgboost,postprocessed,0.632502,0.868335,-0.047707,-0.046964


In [15]:
df_not.to_csv("notPost.csv", index=False)


In [16]:
df_post.to_csv("Post.csv", index=False)
